In [1]:
import csv

sen_emobank = []
v1 = []
a1 = []
d1 = []

with open("/Users/shuxialin/Documents/flip/01and03/data/emobank.csv", 'r') as csvfile :
    readers = csv.reader(csvfile)
    for line in readers:
        if line[0] != 'id':
            sen_emobank.append(line[-1])
            v1.append(line[1])
            a1.append(line[2])
            d1.append(line[3])

In [2]:
sentences = []
lines = []
with open("/Users/shuxialin/Documents/flip/01and03/data/se07_test.xml", 'r') as fileobj :
    for line in fileobj :
        lines.append(line)
        
lines = lines[1:]

for line in lines :
    tokens = (line.split('<')[1]).split('>')
    sentences.append(tokens[-1])

sen_se07 = sentences[:1000]

In [3]:
se07_classes = []

with open('/Users/shuxialin/Documents/flip/01and03/data/se07_test.emotions.gold', 'r') as flie :
    for line in flie:
        token = (line.split())[1:]
        a = token.index(max(token))
        se07_classes.append(a)

In [4]:
sentences = []
classes = []
v = []
a = []
d = []
vad = []

for sen in sen_emobank :
    if sen in sen_se07 :
        sentences.append(sen)
        i = sen_emobank.index(sen)
        v.append(v1[i])
        a.append(a1[i])
        d.append(d1[i])
        b = sen_se07.index(sen)
        classes.append(se07_classes[b])

for i in range(len(a)):
    vad1 = []
    vad1.append(v[i])
    vad1.append(a[i])
    vad1.append(d[i])
    vad.append(vad1)

In [55]:
from bert_serving.client import BertClient
import csv
import  numpy

bc = BertClient(check_length=False)

en_train = bc.encode(sentences[:])
numpy.savetxt("se07_feature.csv", en_train, delimiter=",")

KeyboardInterrupt: 

In [16]:
import csv

vad_ = []

with open("/Users/shuxialin/Documents/flip/01and03/data/se07_vad.csv", 'r') as csvfile :
    readers = csv.reader(csvfile)
    for line in readers:
        vad1 = [round(float(line[0]),2),round(float(line[1]),2),round(float(line[2]),2)]
        vad_.append(vad1)

In [17]:
print(len(v),len(a),len(d),len(classes),len(sentences))

940 940 940 940 940


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
X_train,X_test,y_train,y_test=train_test_split(vad,classes,test_size=0.2)

In [21]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics  import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit, cross_val_score
'''
data = pd.read_csv('train.csv')
traindf = data.drop(['id'], axis = 1)
df = pd.get_dummies(traindf.drop('type', axis = 1))
X_train, X_test, y_train, y_test = train_test_split(df, traindf['type'], test_size = 0.2, random_state = 42)
X_train = pd.get_dummies(traindf.drop('type', axis = 1))
y_train = traindf['type''
'''

accuracy_scorer = metrics.make_scorer(metrics.accuracy_score)

# Using GridSearchCV for the RandomForestClassifier with cv = 10
params = {'n_estimators' : [10, 20, 50, 100], 'criterion' : ['gini', 'entropy'],
          'max_depth' : [None, 1, 2, 3, 5], 'max_features' : ['auto', 'sqrt', 'log2', None]}
rf = RandomForestClassifier(random_state = 0)
clf = GridSearchCV(rf, param_grid = params, scoring = accuracy_scorer, cv = 10, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ('Best score of RandomForestClassifier : {}'.format(clf.best_score_))
print ('Best parameters of RandomForestClassifier : {}'.format(clf.best_params_))
print('RandomForestClassifier classification report')
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score of RandomForestClassifier is: '+str(metrics.accuracy_score(y_test, y_pred)))

# Using GridSearchCV for the BaggingClassifier with cv = 10
params = {'n_estimators' : [10, 20, 50, 100], 'max_samples' : [1, 3, 5, 10]}
bag = BaggingClassifier(random_state = 0)
clf = GridSearchCV(bag, param_grid = params, scoring = accuracy_scorer, cv = 10, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ('Best score of BaggingClassifier: {}'.format(clf.best_score_))
print ('Best parameters of BaggingClassifier: {}'.format(clf.best_params_))
#bag_best = BaggingClassifier(max_samples = 5, n_estimators = 20, random_state = 0)
print('BaggingClassifier classification report')
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score of BaggingClassifier is: '+str(metrics.accuracy_score(y_test, y_pred)))

# Using GridSearchCV for the GradientBoostingClassifier with cv = 5
params = {'learning_rate' : [0.05, 0.07, 0.1, 0.3, 0.5], 'n_estimators' : [10, 20, 50, 100, 200], 'max_depth' : [1, 2, 3, 5]}
gbc = GradientBoostingClassifier(random_state = 0)
clf = GridSearchCV(gbc, param_grid = params, scoring = accuracy_scorer, cv = 10, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ('Best score of GradientBoostingClassifier: {}'.format(clf.best_score_))
print ('Best parameters of GradientBoostingClassifier: {}'.format(clf.best_params_))
print('GradientBoostingClassifier calssification report')
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score of GradientBoostingClassifier is: '+str(metrics.accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))

# Using GridSearchCV for the KNN classifier with CV = 5
params = {'n_neighbors' : [3, 5, 10, 20], 'leaf_size' : [10, 20, 30, 50], 'p' : [1, 2, 5],
          'weights' : ['uniform', 'distance'], 'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}
knc = KNeighborsClassifier()
clf = GridSearchCV(knc, param_grid = params, scoring = accuracy_scorer, cv = 10, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ('Best score of KNN: {}'.format(clf.best_score_))
print ('Best parameters of KNN: {}'.format(clf.best_params_))
print('KNN classification report')
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score of KNN is: '+str(metrics.accuracy_score(y_test, y_pred)))


# Using GridSearchCV for SVC with CV = 10
params = {'kernel' : ['linear', 'rbf'], 'C' : [1, 3, 5, 10], 'degree' : [3, 5, 10]}
svc = SVC(probability = True, random_state = 0)
clf = GridSearchCV(svc, param_grid = params, scoring = accuracy_scorer, cv = 10, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ('Best score of SVC: {}'.format(clf.best_score_))
print ('Best parameters of SVC: {}'.format(clf.best_params_))
print('SVC classification report')
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score of SVC is: '+str(metrics.accuracy_score(y_test, y_pred)))

rf_best = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', max_depth = 5, random_state = 0,max_features=None)
bag_best = BaggingClassifier(max_samples = 10, n_estimators = 100, random_state = 0)
gbc_best = GradientBoostingClassifier(n_estimators = 20, max_depth = 2, learning_rate = 0.3, random_state = 0)
lr_best = LogisticRegression(C = 1, penalty = 'l1')
svc_best = SVC(C = 10, degree = 3, kernel = 'linear')
knc_best = KNeighborsClassifier(algorithm='auto', leaf_size=10, n_neighbors= 20, p=5, weights='uniform')

# voting
voting_clf = VotingClassifier(estimators = [('rf', rf_best), ('bag', bag_best), ('gbc', gbc_best), ('lr', lr_best), ('svc', svc_best), ('knc',knc_best)],voting = 'hard')
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
y_pred = voting_clf.predict(X_test)
print ('\nAccuracy score for VotingClassifier is : '+str(voting_clf.score(X_train, y_train)))
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score is: '+str(metrics.accuracy_score(y_test, y_pred)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best score of RandomForestClassifier : 0.39228723404255317
Best parameters of RandomForestClassifier : {'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'n_estimators': 20}
RandomForestClassifier classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.00      0.00      0.00        16
           2       0.19      0.21      0.20        28
           3       0.37      0.84      0.52        51
           4       0.39      0.43      0.41        37
           5       0.00      0.00      0.00        38

    accuracy                           0.35       188
   macro avg       0.16      0.25      0.19       188
weighted avg       0.21      0.35      0.25       188


Accuracy score of RandomForestClassifier is: 0.34574468085106386


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best score of BaggingClassifier: 0.36569148936170215
Best parameters of BaggingClassifier: {'max_samples': 10, 'n_estimators': 50}
BaggingClassifier classification report
              precision    recall  f1-score   support

           0       0.12      0.06      0.08        18
           1       0.00      0.00      0.00        16
           2       0.25      0.07      0.11        28
           3       0.39      0.73      0.50        51
           4       0.34      0.51      0.41        37
           5       0.25      0.13      0.17        38

    accuracy                           0.34       188
   macro avg       0.22      0.25      0.21       188
weighted avg       0.27      0.34      0.28       188


Accuracy score of BaggingClassifier is: 0.3404255319148936


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

Best score of KNN: 0.3816489361702128
Best parameters of KNN: {'algorithm': 'ball_tree', 'leaf_size': 30, 'n_neighbors': 20, 'p': 2, 'weights': 'uniform'}
KNN classification report
              precision    recall  f1-score   support

           0       0.08      0.06      0.07        18
           1       0.25      0.06      0.10        16
           2       0.08      0.11      0.09        28
           3       0.41      0.63      0.49        51
           4       0.47      0.41      0.43        37
           5       0.16      0.11      0.13        38

    accuracy                           0.30       188
   macro avg       0.24      0.23      0.22       188
weighted avg       0.28      0.30      0.27       188


Accuracy score of KNN is: 0.2978723404255319


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best score of SVC: 0.386968085106383
Best parameters of SVC: {'C': 10, 'degree': 3, 'kernel': 'linear'}
SVC classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.00      0.00      0.00        16
           2       0.11      0.07      0.09        28
           3       0.39      0.82      0.53        51
           4       0.39      0.41      0.40        37
           5       0.16      0.11      0.13        38

    accuracy                           0.34       188
   macro avg       0.18      0.23      0.19       188
weighted avg       0.23      0.34      0.26       188


Accuracy score of SVC is: 0.3351063829787234


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:532: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.

TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'

In [11]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics  import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

accuracy_scorer = metrics.make_scorer(metrics.accuracy_score)

model = xgb.XGBClassifier(nthread=4,
                          learning_rate=0.08,
                          n_estimators=50,
                          max_depth=5,
                          gamma=0,
                          subsample=0.9,
                          colsample_bytree=0.5)


model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = model.predict(X_test)
print ('\nAccuracy score for VotingClassifier is : '+str(model.score(X_train, y_train)))
print (metrics.classification_report(y_test, y_pred))
print ('\nAccuracy score is: '+str(metrics.accuracy_score(y_test, y_pred)))

AttributeError: 'list' object has no attribute 'shape'